In [8]:
import pandas as pd
import numpy as np
import MySQLdb
import datetime

In [20]:
company_order = pd.read_csv('myTempCompanyAll20170915_004.csv',encoding='gbk',header=None,index_col=None)

In [35]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [23]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set('gbk')

In [160]:
V_info_create()
#B_info_create()
#S_info_create()

In [21]:
db.close()

In [36]:
contract_list={'pp':'pp','jm':'jm','jd':'jd','fb':'fb','cs':'cs','bb':'bb','y':'y','v':'v','p':'p'
             ,'m':'m','l':'l','j':'j','i':'i','c':'c','a':'a_1','b':'a_2'}

contract_id=['pp','jm','jd','fb','cs','bb','y','v','p','m','l','j','i','c','a','b']

for x in range(len(contract_id)):
    databaseName1 = contract_list[contract_id[x]] + '_oi_data'
    databaseName2 = contract_list[contract_id[x]] + '_oi_original_data'
    code = contract_list[contract_id[x]]

    cursor = db.cursor()

    # sql = "CREATE DATABASE IF NOT EXISTS " + databaseName1
    # cursor.execute(sql)

    # sql = "CREATE DATABASE IF NOT EXISTS " + databaseName2
    # cursor.execute(sql)

#     create_futures_member(databaseName1)
#     create_non_futures_member(databaseName1)
#     create_total_volume(databaseName1)

    now_date = datetime.datetime(2017,11,29)
    end_date = datetime.datetime(2017,12,27)

    while now_date < end_date:
        split_date = now_date.strftime('%Y-%m-%d').split('-')
        table_date = split_date[0]+split_date[1]+split_date[2]
        
        try:
            data = pd.read_csv('../各交易所期貨持倉量/大商所/'+contract_id[x]+'/'+table_date+'.csv',encoding='gbk',engine='python')
            for i in range(3,len(data)-1):
                for j in range(12):
                    if(data.loc[i][j] == "-"):
                        data.loc[i][j] = None                
                    elif( (j==2 or j==3 or j==6 or j==7 or j==10 or j==11) and data.loc[i][j] != None):
                        data.loc[i][j] = data.loc[i][j].replace(',','')

            total_insert(data,table_date,databaseName2,code)
            futures_member_insert(data,table_date,databaseName1)
            non_futures_member_insert(data,table_date,databaseName1)
            total_volume_insert(data,table_date,databaseName1)
            
            now_date = now_date + datetime.timedelta(days = 1)
            
        except Exception as e:
            now_date = now_date + datetime.timedelta(days = 1)
            print (e)

        #V_info_insert(data,table_date,databaseName)
        #B_info_insert(data,table_date,databaseName)
        #S_info_insert(data,table_date,databaseName)


[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/pp/20171202.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/pp/20171203.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/pp/20171209.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/pp/20171210.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/pp/20171216.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/pp/20171217.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/pp/20171223.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/pp/20171224.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/jm/20171202.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/jm/20171203.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/jm/20171209.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/jm/20171210.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/jm/20171216.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/jm/201712

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/a/20171216.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/a/20171217.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/a/20171223.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/a/20171224.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/b/20171202.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/b/20171203.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/b/20171209.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/b/20171210.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/b/20171216.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/b/20171217.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/b/20171223.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/b/20171224.csv'


In [1]:
def total_insert(data,table_date,databaseName,code):       
    cursor = db.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + "." + code + "_" + table_date+ "(\
    Rank INT(5),\
    Member CHAR(10),\
    Volume FLOAT(5),\
    Increment FLOAT(5),\
    Rank_1 INT(5),\
    Member1 CHAR(10),\
    Buy_Volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Rank_2 INT(5),\
    Member2 CHAR(10),\
    Sales_Volume FLOAT(5),\
    Increment_2 FLOAT(5) )"

    cursor.execute(sql)
    value=[]

    for i in range(3,len(data)-1):      
        value.append((data.loc[i][0],data.loc[i][1],data.loc[i][2],data.loc[i][3],data.loc[i][4],data.loc[i][5],data.loc[i][6],data.loc[i][7],data.loc[i][8],data.loc[i][9],data.loc[i][10],data.loc[i][11]))

    sql = "INSERT INTO " + databaseName + "." + code + "_" + table_date +" VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"        
    cursor.executemany(sql,value)
    db.commit()
    cursor.close()

# 期貨公司成員輸入

In [11]:
def create_futures_member(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".futures_member (\
    Date DATE PRIMARY KEY,\
    Total_volume FLOAT(5),\
    Increment FLOAT(5),\
    Total_buy_volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Total_sales_volume FLOAT(5),\
    Increment_2 FLOAT(5) ) "

    cursor.execute(sql)

In [2]:
def futures_member_insert(data,table_date,databaseName):
    cursor = db.cursor()
    
    sql = "INSERT INTO " + databaseName + ".futures_member VALUES( "\
    + table_date +","\
    + data.loc[0][1].replace(',','') +","\
    + data.loc[0][2].replace(',','') +","\
    + data.loc[0][3].replace(',','') +","\
    + data.loc[0][4].replace(',','') +","\
    + data.loc[0][5].replace(',','') +","\
    + data.loc[0][6].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()
    cursor.close()

# 非期貨公司成員輸入

In [12]:
def create_non_futures_member(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".none_futures_member (\
    Date DATE PRIMARY KEY,\
    Total_volume FLOAT(5),\
    Increment FLOAT(5),\
    Total_buy_volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Total_sales_volume FLOAT(5),\
    Increment_2 FLOAT(5) ) "

    cursor.execute(sql)

In [3]:
def non_futures_member_insert(data,table_date,databaseName):
    cursor = db.cursor()
    sql = "INSERT INTO " + databaseName +".none_futures_member VALUES("\
    + table_date + ","\
    + data.loc[1][1].replace(',','') +","\
    + data.loc[1][2].replace(',','') +","\
    + data.loc[1][3].replace(',','') +","\
    + data.loc[1][4].replace(',','') +","\
    + data.loc[1][5].replace(',','') +","\
    + data.loc[1][6].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()
    cursor.close()

# 總計

In [13]:
def create_total_volume(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume ( Date DATE PRIMARY KEY, \
    V_Valume FLOAT(5),V_Increment FLOAT(5),\
    B_Valume FLOAT(5),B_Increment FLOAT(5),\
    S_Valume FLOAT(5),S_Increment FLOAT(5))"

    cursor.execute(sql)

In [4]:
def total_volume_insert(data,table_date,databaseName):
    cursor = db.cursor()
    total_len = len(data)-1
    sql = "INSERT INTO " + databaseName + ".total_volume VALUES("\
    + table_date + ","\
    + data.loc[total_len][2].replace(',','') +","\
    + data.loc[total_len][3].replace(',','') +","\
    + data.loc[total_len][6].replace(',','') +","\
    + data.loc[total_len][7].replace(',','') +","\
    + data.loc[total_len][10].replace(',','') +","\
    + data.loc[total_len][11].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()

# 各公司資訊

## V名次、成交量、增減

In [45]:
def V_info_create():
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)
    cursor.close()

In [72]:
def V_info_insert(data,table_date,databaseName):
    cursor = db.cursor()
    
    empty = ""
    empty = empty + table_date


    sql = "INSERT INTO " + databaseName + ".v_rank (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
        if(data.loc[i][1] == 'NULL'):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]) + "=" + data.loc[i][0] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][1] == 'NULL'):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]) + "=" + data.loc[i][2] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][1] == 'NULL'):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + data.loc[i][1] + "=" + data.loc[i][3] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()
    
    cursor.close()

## B名次、買單量、增減

In [198]:
def B_info_create():
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

0

In [199]:
def B_info_insert(data,table_date,list_date,databaseName)
    name=""

    empty = ""
    empty = empty + table_date 
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
    if(data.loc[i][1] == "NULL"):
        continue  
        
    sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][5]) + "=" + data.loc[i][4] + " WHERE Date='" + list_date + "'"
    cursor.execute(sql)
    db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][5]) + "=" + data.loc[i][6] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".b_increment SET " + data.loc[i][5] + "=" + data.loc[i][7] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    cursor.close()

## S名次、賣單量、增減

In [203]:
def S_info_create():
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

0

In [204]:
def S_info_insert(data,table_date,list_date,databaseName):
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ','
    name = name + company_order.loc[len(company_order)-1][0]

    empty = ""
    empty = empty + table_date + ","
    for i in range(len(company_order)-1):
        empty = empty + "NULL" + ','
    empty = empty + "NULL"
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
    if(data.loc[i][1] == "NULL"):
        continue  
        
    sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][1]) + "=" + data.loc[i][0] + " WHERE Date='" + list_date + "'"
    cursor.execute(sql)
    db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][1]) + "=" + data.loc[i][2] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == "NULL"):
            continue
        sql = "UPDATE " + databaseName +".s_increment SET " + data.loc[i][9] + "=" + data.loc[i][3] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    cursor.close()